# PP1: Project 1 - pLM-Enhanced Pfam Classification
## 1. Data Loading & preprocessing


### Imports

In [ ]:
#!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
# achtung alles von chatty
import os
import json
import requests
from pathlib import Path
from tqdm import tqdm
#from Bio import SeqIO
from transformers import T5Tokenizer, T5EncoderModel
import torch

### Config (save files to drive)

In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# Choose where to save
SAVE_DIR = Path("/content/drive/MyDrive/protT5_project")  # persisted
# SAVE_DIR = Path("/content/tmp")  # temporary local storage

RAW_DIR = SAVE_DIR / "raw"
PROC_DIR = SAVE_DIR / "processed"
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Get Pfam entries

In [ ]:
def save_raw_data(data_dict):
    for acc, data in data_dict.items():
        fasta = f">{acc}\n{data['sequence']}\n"
        with open(RAW_DIR / f"{acc}.fasta", "w") as f:
            f.write(fasta)
    with open(RAW_DIR / "domain_boundaries.json", "w") as f:
        json.dump(data_dict, f, indent=2)

In [ ]:
def fetch_first_family_accession():
    url = "https://www.ebi.ac.uk/interpro/api/entry/pfam/?type=family"
    response = requests.get(url)
    if response.status_code == 200:
        # simply get the first family accession
        return response.json()['results'][0]['metadata']['accession']
    else:
        raise RuntimeError(f"Pfam family fetch failed: {response.status_code}")

def fetch_sequence_uniprot(uniprot_acc):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_acc}.fasta"
    response = requests.get(url)
    if response.status_code == 200:
        lines = response.text.strip().split('\n')
        return ''.join(lines[1:])  # Skip FASTA header
    else:
        print(f"Failed to get sequence for {uniprot_acc}")
        return None

def fetch_domain_occurrences(family_accession):
    url = f"https://www.ebi.ac.uk/interpro/api/protein/uniprot/entry/pfam/{family_accession}/"
    result = {}
    response = requests.get(url)
    if response.status_code != 200:
        raise RuntimeError(f"Protein domain fetch failed: {response.status_code}")

    proteins_data = response.json()
    for protein in proteins_data["results"]:
        accession = protein['metadata']['accession']
        sequence = fetch_sequence_uniprot(accession)
        if not sequence:
            continue

        entries = protein.get('entries', [])
        domain_regions = []
        for entry in entries:
            for loc in entry.get('entry_protein_locations', []):
                for frag in loc.get('fragments', []):
                    domain_regions.append((frag['start'], frag['end']))
        result[accession] = {"sequence": sequence, "domains": domain_regions}
    return result

Execute

In [ ]:
    family = fetch_first_family_accession()  # or use "PF00012" etc.
    print(f"Using Pfam family: {family}")
    data = fetch_domain_occurrences(family)
    save_raw_data(data)
    print(f"Saved {len(data)} sequences and domain boundaries.")

Using Pfam family: PF00012
Saved 20 sequences and domain boundaries.


In [ ]:
data

{'A0A009HLQ3': {'sequence': 'MAKIIGIDLGTTNSCVAVLEGDKVKVIENAEGARTTPSIIAYKDGEILVGQSAKRQAVTNPKNTLFAIKRLIGRRYEDQAVQKDIGLVPYKIIKADNGDAWVEVNDKKLAPQQISAEILKKMKKTAEDYLGETVTEAVITVPAYFNDAQRQATKDAGKIAGLDVKRIINEPTAAALAFGMDKKEGDRKVAVYDLGGGTFDVSIIEIADLDGDQQIEVLSTNGDTFLGGEDFDNALIEYLVEEFKKEQNVNLKNDPLALQRLKEAAEKAKIELSSSNATEINLPYITADATGPKHLVINVTRAKLEGLVADLVARTIEPCKIALKDAGLSTSDISDVILVGGQSRMPLVQQKVQEFFGREPRKDVNPDEAVAIGAAIQGAVLSGDKNDVLLLDVTPLTLGIETMGGVLTPIIEKNTTIPAKKSQVFSTAADNQPAVDISVYQGERKMAQQNKLLGNFQLGDIPPAPRGVPQIEVSFDINADGILKVSAKDKSTGKEQSIQIKANSGLSDAEIEAMIKDAEANAEEDRKFEELAKARNEADALISSSNKAVKDLGDKVTEDEKTAVNTAVSELEAATKENDVEAIKAKTEALQNILMPITQRAYEQAQQAGGAEGFDPNAFQGGDAGQQKADDGVVDAEFTEVKDDKK',
  'domains': [(4, 602)]},
 'A0A009HXA0': {'sequence': 'MALLQIAEPGQSSAPHEHRIAIGIDLGTTHSLVATVLSGKPKVLNDDKERRLLPSIVHYGNDVTHYGEEAKPFIIADPKNTIVSVKRFMGRSKADIKFQHPYELVGSENEMPAFETRSGRKTPVEISAEILKQLKERAESSLRNPVNGAVITVPAYFDEAQRQATRDAAQLAGLNILRLLNEPTAAAVAYGLDQETNLATDHNYVIYDLGGGTFDVSILRFSQGVFEVLATGGHTALGGDDLDRLIVKWAKKQLNIDTLSDENYAVF

### old

In [ ]:
def get_pfam_entries(pfam_id, max_proteins=20):
    #url = f"https://www.ebi.ac.uk/interpro/api/entry/pfam/{pfam_id}/protein/entry_protein_location/"
    url = "https://www.ebi.ac.uk/interpro/api/entry/pfam/?type=family"
    headers = {'Accept': 'application/json'}
    proteins = []
    while url and len(proteins) < max_proteins:
        r = requests.get(url, headers=headers)
        if not r.ok:
            break
        data = r.json()
        proteins.extend(data['results'])
        url = data.get('next')
    return proteins[:max_proteins]

def download_uniprot_sequence(uniprot_ac):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_ac}.fasta"
    r = requests.get(url)
    if r.ok:
        return str(r.text)
    return None

def save_sequences_and_domains(families):
    seq_records = {}
    domain_info = {}
    for pfam_id in families:
        entries = get_pfam_entries(pfam_id)
        for entry in tqdm(entries, desc=f"Processing {pfam_id}"):
            acc = entry['metadata']['accession']
            for protein in entry['entry_protein_locations']:
                uid = protein['protein']['accession']
                if uid not in seq_records:
                    seq = download_uniprot_sequence(uid)
                    if seq:
                        fasta_path = RAW_DIR / f"{uid}.fasta"
                        fasta_path.write_text(seq)
                        seq_records[uid] = seq
                for frag in protein['fragments']:
                    domain_info.setdefault(uid, []).append({
                        'pfam_id': pfam_id,
                        'start': frag['start'],
                        'end': frag['end']
                    })
    with open(RAW_DIR / "domain_mapping.json", "w") as f:
        json.dump(domain_info, f, indent=2)

### Compute Embeddings

In [ ]:
#!pip install torch transformers tqdm

In [ ]:
import torch
import json
import re
from transformers import T5Tokenizer, T5EncoderModel
from tqdm import tqdm
from pathlib import Path
from torch.nn.utils.rnn import pad_sequence

def load_prostt5():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #Rostlab/prot_t5_xl_half_uniref50-enc
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc", do_lower_case=False, legacy=True)
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")

    if device.type == "cuda":
        print("Moving model to GPU")
        model = model.half()
    model = model.to(device)
    model.eval()
    return tokenizer, model, device

def batch_encode_and_label_sequences(domain_file, batch_size=8):
    tokenizer, model, device = load_prostt5()
    with open(domain_file) as f:
        proteins = json.load(f)

    items = list(proteins.items())
    for i in tqdm(range(0, len(items), batch_size), desc="Batch embedding"):
        batch = items[i:i + batch_size]
        sequences, metadata = [], []

        for acc, data in batch:
            raw_seq = re.sub(r"[UZOB]", "X", data["sequence"])
            if len(raw_seq) > 1022:
                print(f"Skipping {acc}: too long")
                continue

            seq = "<AA2fold> " + " ".join(list(raw_seq))
            sequences.append(seq)
            metadata.append((acc, data["domains"], len(seq.split())))

        if not sequences:
            continue

        tokens = tokenizer.batch_encode_plus(
            sequences, return_tensors="pt", padding=True, add_special_tokens=True
        ).to(device)

        with torch.no_grad():
            output = model(**tokens).last_hidden_state.float().cpu()

        for j, (acc, domains, L) in enumerate(metadata):
            emb = output[j, 1:L+1]  # Remove prefix token and padding
            labels = torch.zeros(emb.shape[0], dtype=torch.long)
            for start, end in domains:
                labels[start:end+1] = 1

            torch.save(emb, PROC_DIR / f"{acc}_embedding.pt")
            torch.save(labels, PROC_DIR / f"{acc}_labels.pt")

Execute

In [ ]:
    family = fetch_first_family_accession()
    print(f"Using Pfam family: {family}")

    data = fetch_domain_occurrences(family)
    save_raw_data(data)
    print(f"Saved {len(data)} sequences and domain boundaries.")

Using Pfam family: PF00012
Saved 20 sequences and domain boundaries.


In [ ]:
    batch_encode_and_label_sequences(RAW_DIR / "domain_boundaries.json")
    print("Done.")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

Moving model to GPU


Batch embedding: 100%|██████████| 3/3 [00:08<00:00,  2.74s/it]

Done.


In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pathlib import Path

# ---- FIND ALL EMBEDDING FILES ----
embedding_files = sorted(PROC_DIR.glob("*_embedding.pt"))

print(f"Found {len(embedding_files)} embedding files.")

# ---- VISUALIZE EACH ----
for i,emb_file in enumerate(embedding_files):
    if i == 5:
      break
    acc = emb_file.stem.replace("_embedding", "")
    label_file = PROC_DIR / f"{acc}_labels.pt"

    if not label_file.exists():
        print(f"Missing label for {acc}, skipping.")
        continue

    emb = torch.load(emb_file)  # (L, 1024)
    labels = torch.load(label_file)  # (L,)

    # PCA projection to 2D
    pca = PCA(n_components=2)
    coords = pca.fit_transform(emb)

    # Plot
    plt.figure(figsize=(7, 5))
    plt.scatter(coords[:, 0], coords[:, 1], c=labels, cmap="coolwarm", s=10, alpha=0.8)
    plt.title(f"Residue Embeddings (PCA): {acc}")
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.colorbar(label="Domain label (0=non-domain, 1=domain)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

#### old

In [ ]:
def compute_prott5_embeddings():
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
    model.eval()
    model = model.to("cuda" if torch.cuda.is_available() else "cpu")

    for fasta_file in RAW_DIR.glob("*.fasta"):
        records = list(SeqIO.parse(fasta_file, "fasta"))
        if not records:
            continue
        seq = str(records[0].seq).replace(" ", "").replace("\n", "")
        ids = tokenizer.batch_encode_plus([seq], add_special_tokens=True, return_tensors="pt")
        ids = {k: v.to(model.device) for k, v in ids.items()}
        with torch.no_grad():
            emb = model(**ids).last_hidden_state.squeeze(0).cpu()
        torch.save(emb, PROC_DIR / f"{fasta_file.stem}_embedding.pt")

def label_residues_and_save():
    with open(RAW_DIR / "domain_mapping.json") as f:
        domain_map = json.load(f)

    for uid, domains in domain_map.items():
        embedding_path = PROC_DIR / f"{uid}_embedding.pt"
        if not embedding_path.exists():
            continue
        emb = torch.load(embedding_path)
        labels = torch.zeros(emb.shape[0], dtype=torch.long)
        for i, domain in enumerate(domains):
            labels[domain['start']:domain['end'] + 1] = i + 1  # Label domain with index
        torch.save(labels, PROC_DIR / f"{uid}_labels.pt")

### DataSet class


In [ ]:
import torch
from pathlib import Path
from torch.utils.data import Dataset

class PfamEmbeddingDataset(Dataset):
    def __init__(self, processed_dir):
        self.processed_dir = Path(processed_dir)
        self.entries = sorted(set(f.stem.replace('_embedding', '') for f in self.processed_dir.glob("*_embedding.pt")))

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        acc = self.entries[idx]
        emb_path = self.processed_dir / f"{acc}_embedding.pt"
        label_path = self.processed_dir / f"{acc}_labels.pt"

        embeddings = torch.load(emb_path)     # Shape: [L, 1024]
        labels = torch.load(label_path)       # Shape: [L]
        return embeddings, labels, acc

In [ ]:
from torch.utils.data import DataLoader

dataset = PfamEmbeddingDataset("data/processed")
loader = DataLoader(dataset, batch_size=1, shuffle=True)

for embeddings, labels, acc in loader:
    print(embeddings.shape, labels.shape, acc)
    break

ValueError: num_samples should be a positive integer value, but got num_samples=0

### Embedding Viz

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

def visualize_embedding(embeddings, labels, method='pca'):
    X = embeddings.numpy()
    y = labels.numpy()

    if method == 'pca':
        reducer = PCA(n_components=2)
    elif method == 'tsne':
        reducer = TSNE(n_components=2, perplexity=30, learning_rate=200)
    else:
        raise ValueError("method must be 'pca' or 'tsne'")

    X_red = reducer.fit_transform(X)

    plt.figure(figsize=(8, 6))
    plt.scatter(X_red[:, 0], X_red[:, 1], c=y, cmap='coolwarm', s=5)
    plt.title(f"{method.upper()} of per-residue embeddings")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.colorbar(label="Residue label (0 = non-domain, 1 = domain)")
    plt.grid(True)
    plt.show()


In [ ]:
# Load data (change path if needed)
dataset = PfamEmbeddingDataset("data/processed")

# Pick one protein
embeddings, labels, acc = dataset[0]
print(f"Visualizing: {acc}, shape: {embeddings.shape}")

# Visualize
visualize_embedding(embeddings, labels, method='pca')

or using UMAP for better structure preservation

In [ ]:
!pip install umap-learn

In [ ]:
import umap

def visualize_umap(embeddings, labels):
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine')
    X = embeddings.numpy()
    y = labels.numpy()
    X_umap = reducer.fit_transform(X)

    plt.figure(figsize=(8, 6))
    plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='coolwarm', s=5)
    plt.title("UMAP of per-residue embeddings")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    plt.colorbar(label="Residue label")
    plt.grid(True)
    plt.show()

In [ ]:
dataset = PfamEmbeddingDataset("data/processed")
# Pick one protein
embeddings, labels, acc = dataset[0]
print(f"Visualizing: {acc}, shape: {embeddings.shape}")
visualize_umap(embeddings, labels)